[Surprise · A Python scikit for recommender systems.](http://surpriselib.com/)

In [1]:
# !pip install scikit-surprise
# !conda install -c conda-forge scikit-surprise

### SVD

In [2]:
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')
data

In [3]:
pd.DataFrame(data.raw_ratings, columns=["user", "movie", "rating", "timestamp"])

,user,movie,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


In [4]:
# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9373  0.9373  0.9373  0.9384  0.9392  0.9379  0.0008  
MAE (testset)     0.7386  0.7394  0.7368  0.7362  0.7421  0.7386  0.0021  
Fit time          3.33    3.31    3.75    3.80    4.00    3.64    0.27    
Test time         0.10    0.12    0.18    0.11    0.15    0.13    0.03    


{'test_rmse': array([0.93733852, 0.93726385, 0.93729749, 0.93844917, 0.93916891]),
 'test_mae': array([0.73859395, 0.7394031 , 0.7368377 , 0.73624211, 0.7420687 ]),
 'fit_time': (3.3259658813476562,
  3.3076460361480713,
  3.754251003265381,
  3.802367925643921,
  3.9969279766082764),
 'test_time': (0.09589290618896484,
  0.12314105033874512,
  0.18082308769226074,
  0.1055443286895752,
  0.1482381820678711)}

## KNN

In [5]:
from surprise import KNNBasic

# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
knn = KNNBasic()
knn.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [6]:
# Run 5-fold cross-validation and print results.
cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9854  0.9704  0.9788  0.9763  0.9801  0.9782  0.0049  
MAE (testset)     0.7818  0.7648  0.7705  0.7708  0.7740  0.7724  0.0056  
Fit time          0.28    0.31    0.31    0.30    0.30    0.30    0.01    
Test time         2.59    2.76    2.23    2.59    2.47    2.53    0.18    


{'test_rmse': array([0.98536731, 0.97039118, 0.97877866, 0.97627604, 0.98007395]),
 'test_mae': array([0.78181766, 0.7648367 , 0.7705138 , 0.77082774, 0.77395511]),
 'fit_time': (0.27544116973876953,
  0.3148651123046875,
  0.30864715576171875,
  0.30131101608276367,
  0.29959821701049805),
 'test_time': (2.5936977863311768,
  2.759399890899658,
  2.2284762859344482,
  2.5923430919647217,
  2.4710888862609863)}

In [7]:
import pandas as pd

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


# Creation of the dataframe. Column names are irrelevant.
ratings_dict = {'itemID': [1, 1, 1, 2, 2],
                'userID': [9, 32, 2, 45, 'user_foo'],
                'rating': [3, 2, 4, 3, 1]}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([1.75177317, 1.78951043]),
 'test_mae': array([1.74232665, 1.51266678]),
 'fit_time': (5.4836273193359375e-05, 2.8133392333984375e-05),
 'test_time': (4.220008850097656e-05, 1.7881393432617188e-05)}